In [1]:
import logging
import os
try:
    from pathlib2 import Path
except ImportError:
    from pathlib import Path

import numpy as np
import torch
import torch.multiprocessing as mp
import parmap

from detect.detector import Detect
from localization_pipeline.denoiser import Denoise

from detect.deduplication import deduplicate_gpu, deduplicate

from scipy.signal import argrelmin

from detect.run import run


In [2]:
geom_path = '/media/cat/julien/spikes_localization_registration/channels_maps/np2_channel_map.npy'
path_nn_detector = '/media/cat/julien/spikes_localization_registration/pretrained_detector/detect_np2.pt'
path_nn_denoiser = '/media/cat/julien/spikes_localization_registration/pretrained_denoiser/denoise.pt'
standardized_path = '/media/cat/julien/nick_drift/different_preprocessings/data_standardized_registered/standardized_unregistered.bin'
standardized_dtype = 'float32'
sampling_rate = 30000
len_recording = 1000
output_directory = 'detection_results_threshold'

geom_array = np.load(geom_path)
apply_nn = False ### If set to false, run voltage threshold instead of NN detector 
spatial_radius = 70
n_sec_chunk = 1
n_processors = 4
n_sec_chunk_gpu_detect = .1
detect_threshold = 6 ## 0.5 if apply NN, 4/5/6 otherwise 
n_filters_detect = [16, 8, 8] 
spike_size_nn = 121 ### In sample steps
n_filters_denoise = [16, 8, 4]
filter_sizes_denoise = [5, 11, 21]
n_batches = len_recording//n_sec_chunk


In [27]:
sampling_rate = 30000

run(standardized_path, standardized_dtype, output_directory, geom_array, spatial_radius, apply_nn, n_sec_chunk, n_batches, n_processors, n_sec_chunk_gpu_detect, sampling_rate, len_recording,
    detect_threshold, path_nn_detector, n_filters_detect, spike_size_nn, path_nn_denoiser, n_filters_denoise, filter_sizes_denoise, run_chunk_sec='full')

   batch length to (sec):  4  (longer increase speed a bit)
   length of each seg (sec):  0.1


/media/cat/julien/spikes_localization_registration/detect/detector.py:80: RuntimeWarning: invalid value encountered in log
  (probs >= temporal_max) & (probs > np.log(threshold / (1 - threshold))))


batch : 10
batch : 11
batch : 12
batch : 13
batch : 14
batch : 15
batch : 16
batch : 17
batch : 18
batch : 19
batch : 20
batch : 21
batch : 22
batch : 23
batch : 24
batch : 25
batch : 26
batch : 27
batch : 28
batch : 29
batch : 30
batch : 31
batch : 32
batch : 33
batch : 34
batch : 35
batch : 36
batch : 37
batch : 38
batch : 39
batch : 40
batch : 41
batch : 42
batch : 43
batch : 44
batch : 45
batch : 46
batch : 47
batch : 48
batch : 49
batch : 50
batch : 51
batch : 52
batch : 53
batch : 54
batch : 55
batch : 56
batch : 57
batch : 58
batch : 59
batch : 60
batch : 61
batch : 62
batch : 63
batch : 64
batch : 65
batch : 66
batch : 67
batch : 68
batch : 69
batch : 70
batch : 71
batch : 72
batch : 73
batch : 74
batch : 75
batch : 76
batch : 77
batch : 78
batch : 79
batch : 80
batch : 81
batch : 82
batch : 83
batch : 84
batch : 85
batch : 86
batch : 87
batch : 88
batch : 89
batch : 90
batch : 91
batch : 92
batch : 93
batch : 94
batch : 95
batch : 96
batch : 97
batch : 98
batch : 99
batch : 10

batch : 728
batch : 729
batch : 730
batch : 731
batch : 732
batch : 733
batch : 734
batch : 735
batch : 736
batch : 737
batch : 738
batch : 739
batch : 740
batch : 741
batch : 742
batch : 743
batch : 744
batch : 745
batch : 746
batch : 747
batch : 748
batch : 749
batch : 750
batch : 751
batch : 752
batch : 753
batch : 754
batch : 755
batch : 756
batch : 757
batch : 758
batch : 759
batch : 760
batch : 761
batch : 762
batch : 763
batch : 764
batch : 765
batch : 766
batch : 767
batch : 768
batch : 769
batch : 770
batch : 771
batch : 772
batch : 773
batch : 774
batch : 775
batch : 776
batch : 777
batch : 778
batch : 779
batch : 780
batch : 781
batch : 782
batch : 783
batch : 784
batch : 785
batch : 786
batch : 787
batch : 788
batch : 789
batch : 790
batch : 791
batch : 792
batch : 793
batch : 794
batch : 795
batch : 796
batch : 797
batch : 798
batch : 799
batch : 800
batch : 801
batch : 802
batch : 803
batch : 804
batch : 805
batch : 806
batch : 807
batch : 808
batch : 809
batch : 810
batc

'detection_results_threshold/spike_index.npy'

In [3]:
#### LOCALIZATION

import os
import numpy as np
from tqdm import tqdm
# from residual import RESIDUAL
from localization_pipeline.localizer import LOCALIZER
from localization_pipeline.merge_results import get_merged_arrays


In [4]:
### Change paths to your data here
bin_file = standardized_path
residual_file = bin_file
dtype_input = 'float32'

fname_spike_train = 'detection_results_threshold/spike_index.npy'
# Sort spike train if not 
spt_array = np.load(fname_spike_train)
spt_array = spt_array[spt_array[:, 0].argsort()]
np.save(fname_spike_train, spt_array)

geom_path = "channels_maps/np2_channel_map.npy"
n_channels = np.load(geom_path).shape[0]

denoiser_weights = path_nn_denoiser
denoiser_min = 42 ## Goes with the weights

n_batches = 1000
len_recording = 1000
sampling_rate = 30000

fname_templates = None

In [ ]:
localizer_obj = LOCALIZER(bin_file, residual_file, dtype_input, fname_spike_train, fname_templates, geom_path, denoiser_weights, denoiser_min)
# localizer_obj.get_offsets()
# localizer_obj.compute_aligned_templates()
localizer_obj.load_denoiser()
output_directory = 'position_results_files'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
for i in tqdm(range(n_batches)):
    localizer_obj.get_estimate(i, threshold = 6, output_directory = 'position_results_files')



  0%|          | 1/1000 [00:11<3:17:49, 11.88s/it]